In [1]:
!pip install huggingface_hub
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install sentence-transformers

^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.0 MB/s eta 0:00:00 0:00:01


In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import huggingface_hub
from langchain.vectorstores import Chroma
from langchain.chains import conversational_retrieval

In [2]:
# Load the pdf file and split it into smaller chunks
loader = PyPDFLoader("./pdf_doc/Compromis n°1 de vente d'immeuble entier BOUILLAGUET _ CHEZE [36442].pdf")
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = CharacterTextSplitter(
    chunk_size=1500, chunk_overlap=0
)  # Le paramètre chunk_overlap détermine le nombre de caractères qui se chevauchent entre deux segments de texte consécutifs lors du découpage du texte en morceaux (chunks).
texts = text_splitter.split_documents(documents)

In [ ]:
# We will use HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

/home/jbcheze/.cache/pypoetry/virtualenvs/ov-project-pLFMpO-6-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/jbcheze/.cache/pypoetry/virtualenvs/ov-project-pLFMpO-6-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Using Chroma vector database to store and retrieve embeddings of our text
db = Chroma.from_documents(texts, embeddings) # Creation de la base de donné vectorielle chroma 
retriever = db.as_retriever(search_kwargs={"k": 2}) # retriever sert à faire des recherches sémantiques.

: 

In [28]:
import os
from dotenv import load_dotenv

In [29]:
# We are using Mistral-7B for this question answering
from langchain_community.llms import VLLMOpenAI
load_dotenv()
mixtral_api_key = os.getenv("MIXTRAL_API_KEY")
MIXTRAL_API_BASE = os.getenv("MIXTRAL_API_BASE")
huggingfacehub_api_token = os.getenv("huggingfacehub_api_token")

llm = VLLMOpenAI(
        openai_api_key=mixtral_api_key,
        openai_api_base=MIXTRAL_API_BASE,
        model_name="mistralai/Mixtral-8X7B-Instruct-v0.1",
        temperature=0.0,
        max_tokens=7000,
    )

In [30]:
from langchain.chains import ConversationalRetrievalChain

# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever, return_source_documents=True
)

In [31]:
# We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys

chat_history = []
while True:
    query = input("Prompt: ")
    # To exit: use 'exit', 'quit', 'q', or Ctrl-D.",
    if query.lower() in ["exit", "quit", "q"]:
        print("Exiting")
        sys.exit()
    result = qa_chain({"question": query, "chat_history": chat_history})
    print("Answer: " + result["answer"] + "\n")
    chat_history.append((query, result["answer"]))

Answer:  The bien is located in Aubazine, France, specifically at 2, rue Raymond Bouillaguet.

Exiting


SystemExit: 

/home/jbcheze/.cache/pypoetry/virtualenvs/ov-project-pLFMpO-6-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [34]:
prompt = (
    f"""
    Je souhaite que tu me rédiges un résumé du document qui sera organisé selon les points suivants:
    - section "Coordonnées du Vendeur et de l'Acquéreur" 
    - section "Bien Immobilier" 
    - section "Hypothèque et servitudes"
    - section "Dossier du Diagnostic Technique (DDT)
    - section "Montant et Modalités de Paiement"
    - section "Durée de Validité de la Promesse de Vente et Date Limite de Signature de l'Acte de Vente Définitif"
    - section "Montant de l'Indemnité d'Immobilisation et Conditions Suspensives"
""",

    # Tu rempliras chaque section avec les informations adéquates en organisant ton compte-rendu en bullet points.
    
)

# prompt = (f"Résume moi le document")

result = qa_chain({"question": prompt, "chat_history": chat_history})
print("Answer: " + result["answer"] + "\n")

Answer: 

A summary of the document should include the following points:

1. The location and description of the property, including the number of rooms and the presence of a garage.
2. The cadastral references and the total area of the property.
3. The fact that the property is being sold with all its dependencies, servitudes, and mitoyenneties.
4. The fact that the buyer has already viewed and is familiar with the property.
5. The agreement between the parties that the buyer will visit the property with the seller before the final sale to confirm its condition and the seller's departure.
6. The fact that no furniture or fixtures are included in the sale.
7. The plan of the property attached to the document.

